Given: $N \sim Poisson(\lambda)$ and $X_1, \dots, X_n \sim \vec{\pi}$

$X_k(t)$ is continous time MC with $X_k(0) = X_k$
$N_t(a) = $\{k:X_k(t) = a\}$

i.e. $N_t$ is the number of visits to state $a$ in time $t$.

$\sum_a\pi(a)Q_{ab}=0$ for each $b$ with the constraint $\sum_a\pi(a)=1$


$\sum_a\pi(a)Q_{ab}=0$ $\implies$ $\vec{\pi}^TQ=0$ $\implies$ 

$$
\begin{align*}
\vec{\pi}^TQ&=0\\
\Longleftrightarrow \vec{\pi}^TQ^n&=0\ \  \forall n \geq 1\\ 
\Longleftrightarrow \sum_{n\geq 1}\vec{\pi}\frac{t^n}{n!}Q^n &=0 \ \  \forall t \geq 0\\
\Longleftrightarrow \vec{\pi}\sum_{n\geq 0}\frac{t^n}{n!}Q^n &=\vec{\pi}\\
\Longleftrightarrow \vec{\pi}P &=\vec{\pi}\\
\Longleftrightarrow \vec{\pi}\  \text{is a stationary distribution}
\end{align*}
$$

Now, $P(X_k(t)=a)=\pi(a)$ and $N_t(a) = \{k:X_k(t) = a\}$ $\implies$ $N_t(a)|N \sim Binom(N, \pi(a))$ and 
$N \sim Poisson(\lambda)$ then $N_t \sim Poisson(\lambda \pi)$ [We did this in class TODO]

## Problem 2

In [1]:
%matplotlib inline
from __future__ import division
import pandas as pd
import matplotlib
import itertools
matplotlib.rcParams['figure.figsize'] = (16,12)
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1)

def propose(S):
    r = np.random.choice(len(S), 2)
    rs = np.sort(r)
    j,k=rs[0],rs[1]
    y=np.copy(S)
    y[j:k+1] = y[j:k+1][::-1]
    return y

def count_cycles(S):
    sample_length = len(S)
    n_cycles = 0
    index = 0
    length_travelled = 0
    visited = []
    while length_travelled < sample_length:
        if S[index] == index and index < sample_length :
            index+=1
            n_cycles+=1
            length_travelled+=1
        else:
            visited.append(index)
            index = S[index]
            length_travelled+=1
            if index not in visited:
                n_cycles+=1
    return n_cycles

In [2]:
N = [2,3,4, 100]
alpha = 3

In [3]:
assert count_cycles([0,1]) == 2
assert count_cycles([0,2,1]) == 2
assert count_cycles([1,0]) == 1

In [11]:
N_iterations = 10000

def theoretical(S, alpha):
    n_cycles = count_cycles(S)
    return n_cycles**alpha/sum([count_cycles(x)**alpha  for x in itertools.permutations(S)])


def run(n):
    oldS = np.arange(n)
    old_n_cycles = count_cycles(oldS)
    count_dict = {}
    for i in range(N_iterations):
        proposedS = propose(oldS)
        new_n_cycles = count_cycles(proposedS)
        pi_ab = new_n_cycles**alpha/(old_n_cycles**alpha)
        q = min(1,pi_ab)
        if q>= np.random.uniform():
            oldS = proposedS
            old_n_cycles = new_n_cycles
        tkey = ','.join([str(x+1) for x in oldS.tolist()])
        key="["+tkey+"]"
        if key not in count_dict:
            count_dict[key] = [0,0,0]
            count_dict[key][1] = theoretical(oldS,alpha)
            count_dict[key][2] = old_n_cycles
        count_dict[key][0]+=1
    df = pd.DataFrame(count_dict)
    df=df.transpose()
    df.columns=[r'Simulated $\pi(s)$', 'Theoretical', 'c(s)']
    df[r'Simulated $\pi(s)$'] = df[r'Simulated $\pi(s)$']/N_iterations
    df['Percentage Error'] = 100*(df[r'Simulated $\pi(s)$']/df['Theoretical']-1)
    df.index.name='State'
    return df


## n=2

In [12]:
df  = run(N[0])
df

,Simulated $\pi(s)$,Theoretical,c(s),Percentage Error
State,,,,
"[1,2]",0.8894,0.888889,2,0.0575
"[2,1]",0.1106,0.111111,1,-0.4600


## n=3


In [13]:
df = run(N[1])
df

,Simulated $\pi(s)$,Theoretical,c(s),Percentage Error
State,,,,
"[1,2,3]",0.5174,0.509434,3,1.563704
"[1,3,2]",0.1541,0.150943,2,2.091250
"[2,1,3]",0.0173,0.018868,1,-8.310000
"[2,3,1]",0.1362,0.150943,2,-9.767500
"[3,1,2]",0.1614,0.150943,2,6.927500
"[3,2,1]",0.0136,0.018868,1,-27.920000


## n=4

In [14]:
count_dict = run(N[2])
count_dict

,Simulated $\pi(s)$,Theoretical,c(s),Percentage Error
State,,,,
"[1,2,3,4]",0.1805,0.169761,4,6.325781
"[1,2,4,3]",0.0678,0.071618,3,-5.331111
"[1,3,2,4]",0.0211,0.021220,2,-0.566250
"[1,3,4,2]",0.0729,0.071618,3,1.790000
"[1,4,2,3]",0.0751,0.071618,3,4.861852
"[1,4,3,2]",0.0259,0.021220,2,22.053750
"[2,1,3,4]",0.0020,0.002653,1,-24.600000
"[2,1,4,3]",0.0016,0.002653,1,-39.680000
"[2,3,1,4]",0.0268,0.021220,2,26.295000


## N=100

In [ ]:
df  = run(N[3])
#df
expectation = sum(df[r'Simulated $\pi(s)$']*df['c(s)'])
expectation2 = sum(df[r'Simulated $\pi(s)$']*df['c(s)']*df['c(s)'])


In [ ]:
print expectation, expectation2

In [ ]:
print np.mean(df['c(s)'])

$\sum_{s \in S_a}\pi(s)c(s)=E[c(s)]$

and similarly, 

$\sum_{s \in S_a}\pi(s)c^2(s)=E[c^2(s)]=Var(c(s))+E^2[c(s)]$

In [ ]:
cycles = df['c(s)']
plt.hist(cycles, normed=True)

## Problem 3

In [ ]:
N = 1000
chrom_length = 3*(10**9)
transposon_length = 3*1000
mu = 0.05
t_positions = []

x_initial = np.random.uniform(0,chrom_length,size=1)
for i in range(N):
    temp_mu = np.random.uniform(0,1,1)
    if temp_mu <=0.05:
        ## Tranpose
        ## Ignore overlap with existing region
        y,z = np.random.uniform(0, chrom_length,size=2)
        